In [24]:
import pandas as pd
from nba_api.stats.endpoints import playercareerstats, commonplayerinfo, playerprofilev2
from nba_api.stats.static import players
from nba_api.stats.static import teams

In [2]:
df = pd.read_csv('all_players.csv')

In [ ]:
nba_players = [i for i in players.get_players() if i['is_active']==True]
teams_df = pd.DataFrame(teams.get_teams())
players_df = pd.DataFrame(nba_players)

In [13]:
latest_df = df[df['SEASON_ID'] == '2021-22']
latest_df['AST_per_game'] = latest_df['AST']/latest_df['GP']
latest_df['REB_per_game'] = latest_df['REB']/latest_df['GP']
latest_df['PTS_per_game'] = latest_df['PTS']/latest_df['GP']

C:\Users\gabri\AppData\Local\Temp\ipykernel_13920\3656711522.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latest_df['AST_per_game'] = latest_df['AST']/latest_df['GP']
C:\Users\gabri\AppData\Local\Temp\ipykernel_13920\3656711522.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latest_df['REB_per_game'] = latest_df['REB']/latest_df['GP']
C:\Users\gabri\AppData\Local\Temp\ipykernel_13920\3656711522.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

In [14]:
latest_df.tail()

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,REB,AST,STL,BLK,TOV,PF,PTS,AST_per_game,REB_per_game,PTS_per_game
3307,201152,2021-22,0,0,TOT,33.0,52,1,845,141,...,208,104,54,18,52,81,322,2.000000,4.000000,6.192308
3311,1629027,2021-22,0,1610612737,ATL,23.0,76,76,2652,711,...,284,737,72,7,303,128,2155,9.697368,3.736842,28.355263
3312,1630209,2021-22,0,1610612748,MIA,23.0,56,12,706,130,...,294,49,17,20,41,84,299,0.875000,5.250000,5.339286
3321,203469,2021-22,0,1610612757,POR,29.0,27,0,355,51,...,125,22,8,6,19,56,140,0.814815,4.629630,5.185185
3329,1627826,2021-22,0,1610612746,LAC,25.0,76,76,1852,310,...,644,120,36,77,114,203,785,1.578947,8.473684,10.328947


In [16]:
latest_df.sort_values('PTS_per_game', ascending=False).head()

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,REB,AST,STL,BLK,TOV,PF,PTS,AST_per_game,REB_per_game,PTS_per_game
934,203954,2021-22,0,1610612755,PHI,28.0,68,68,2296,666,...,796,284,77,99,214,181,2079,4.176471,11.705882,30.573529
1708,2544,2021-22,0,1610612747,LAL,37.0,56,56,2084,640,...,459,349,73,59,196,121,1695,6.232143,8.196429,30.267857
68,203507,2021-22,0,1610612749,MIL,27.0,67,67,2204,689,...,778,388,72,91,219,212,2002,5.791045,11.611940,29.880597
904,201142,2021-22,0,1610612751,BKN,33.0,55,55,2047,578,...,407,351,48,52,191,113,1643,6.381818,7.400000,29.872727
828,1629029,2021-22,0,1610612742,DAL,23.0,65,65,2301,641,...,593,568,75,36,292,145,1847,8.738462,9.123077,28.415385


In [131]:
dwhite_id = 1628401
grouped = latest_df.groupby(by=['PLAYER_ID','TEAM_ID'], as_index=False).agg({'PTS_per_game':'mean',
                                         'AST_per_game':'mean',
                                        'REB_per_game':'mean',
                                                  'GP':'sum'}).sort_values(
                    by=['TEAM_ID','GP'], ascending=[False, False]).drop_duplicates(subset=['PLAYER_ID'], keep='first')
grouped.head()

,PLAYER_ID,TEAM_ID,PTS_per_game,AST_per_game,REB_per_game,GP
351,1628970,1610612766,20.162500,3.750000,6.987500,80
206,1626162,1610612766,14.960526,1.105263,3.973684,76
526,1630163,1610612766,20.106667,7.613333,6.680000,75
134,203486,1610612766,6.534247,3.095890,7.739726,73
218,1626179,1610612766,19.287671,4.465753,4.315068,73


In [132]:
grouped = grouped.merge(players_df[['id','full_name']], left_on='PLAYER_ID',right_on='id')
grouped = grouped.merge(teams_df[['id','full_name']], left_on='TEAM_ID', right_on='id')
grouped.drop(columns=['id_x','id_y'], inplace=True)
grouped.rename(columns={'full_name_x':'full_name_player', 'full_name_y':'full_name_team'}, inplace=True)

In [133]:
grouped.head()

,PLAYER_ID,TEAM_ID,PTS_per_game,AST_per_game,REB_per_game,GP,full_name_player,full_name_team
0,1628970,1610612766,20.162500,3.750000,6.987500,80,Miles Bridges,Charlotte Hornets
1,1626162,1610612766,14.960526,1.105263,3.973684,76,Kelly Oubre Jr.,Charlotte Hornets
2,1630163,1610612766,20.106667,7.613333,6.680000,75,LaMelo Ball,Charlotte Hornets
3,203486,1610612766,6.534247,3.095890,7.739726,73,Mason Plumlee,Charlotte Hornets
4,1626179,1610612766,19.287671,4.465753,4.315068,73,Terry Rozier,Charlotte Hornets


In [136]:
grouped.to_csv('../db/initial_migration/players_202122.csv', index=False)